Assalam walaikum Aqeel,
Hope you're doing good. My name's Arif and we had talked about your work on animation of random walks of pi digits. Well, not really talked but just a one message conversation where I had asked about the libraries you used to work on it.

I was intrigued by the visualization and wanted to execute it myself from scratch and so I had started doing it, but then the semester resumed after spring break and then things happened due to which I was not able to complete it.

I started coding it again and I got stuck at some point. Can you please help me understand where I am going wrong and whether the approach that I am taking is correct or not? I wouldn't mind a long message in response but would really appreciate if we could hop on a call and resolve this and also get to know each other, I also wanted your guidance in the field of Machine Learning and would love to hear what it feels like being in your shoes as an ML Engineer at NVIDIA.
Looking forward to hearing from you and having a great conversation.

So, here's the approach that I took.

Firstly importing all the libraries:



In [1]:
# to install celluloid (for animations and understanding the sequences of random walks)
!pip install celluloid

# Cell to import all the libraries necessary for the execution of the necessary code snippets. 

import matplotlib.pyplot as plt
import numpy as np
import pandas as pnd
import math

from celluloid import Camera as Cam
from numpy import random 
from IPython.display import HTML, clear_output
from google.colab import output

plt.style.use('dark_background')

from google.colab import drive
drive.mount('/content/drive')

dataframe =  pnd.read_csv("/content/drive/MyDrive/Personal Projects/Random Walks/pi_100k_digits.csv", encoding="cp1252", skip_blank_lines=True)
data = dataframe["digits"].to_numpy().reshape(-1,1)
data.shape

Mounted at /content/drive


(100000, 1)

So, the approach that I took is, I considered that each walk or move happens in 360 degree. So, if a digit 0 comes in, the point will move from its current position, current [x,y] cordinate to another [x,y] cordinate at an angle 36 degree. If digit 1 comes in, it will move at an angle of 72 degree, for digit 2 it will move at an angle of 108. 

So, for every time digit 0 appears, the cordinates will be [ x + (cos(36) * hypotenuse), y + (sin(36) * hypotenuse)] and I assume the hypotenuse to be 1, since we wanna move it at an angle with a unit distance.

Below is the code, with each line annotated.

In [34]:
#Function that formed patterns

def Random_Walks_Line_Plot(instances):

  # instances can be anything here, sequence of pi digits or some random numbers from 1 to 4 or 5 to 8 or any sequence.
  # using some random numbers just for the sake of tracing, but i am more interested in pi digits.
  # 'uniq' contains the unique values of the pi-digits dataset, which is 0 to 9
  uniq = np.unique(instances[:,0]).reshape(-1,1)
  n = len(uniq)

  # angles contains the angle at which the point has to be moved at a unit distance (depending upon how many unique values are there, those many angles are produced)
  angles = np.array([ int((x)*(360/n)) for x in range(len(uniq)) ]).reshape(-1,1)
  
  # uniq_angles is a dictionary that has pairs, value-angle : value can be any digit or number and angle represents the degree in which the movement has to be done.
  uniq_angles = dict(zip(uniq[:,0],angles[:,0]))

  # creating an empty 'x' array consisting of x and y co-ordinates and  Assigning the first point as origin.
  x = np.empty((len(instances)+1,2)) 
  x[0,0], x[0,1] = 0,0

  # celluloid snippet to record the visualisations
  fig, ax = plt.subplots(dpi = 75)
  cam = Cam(fig)
  i = 1

  # iteration loop to go through instances, assign cordinates accordingly and plot them
  for k in instances[:,0]:
    angle = uniq_angles.get(k)

    # move to make from current x,y cordinate
    move = np.array([np.cos(np.radians(angle)),np.sin(np.radians(angle))])

    # move added to the current cordinate OR the new cordinate
    x[i] = x[i] + move
    
    plt.plot(x[0:i,0], x[0:i,1], '.-')
    
    trace = "instance : " + str(i) + " and value : " + str(k)  
    plt.axis('off')
    ax.text(0.1, 1.05, trace, transform=ax.transAxes, fontstyle = 'italic', fontweight = 'bold')
    cam.snap()
    i += 1
  plt.close(fig)
  animation = cam.animate(blit=False, interval=500)
  return HTML(animation.to_html5_video())
  

In [23]:
Random_Walks_Line_Plot(np.random.randint(5, size=100).reshape(-1,1))

In [24]:
Random_Walks_Line_Plot(data[0:100])  

The above two outputs is what I get.

The first one for random numbers and the second one if of first 100 pi digits.

My query is I am not sure why it ends up forming a pattern. I ended up thinking that cordinates shouldn't repeat at all (which in my case is).

So, I modified the above function a bit, as follows in the next block of code.

In the modified version I made a dictionary of cordinates. For each iteration, i.e for each value seen I check if the cordinates for the current value are there in dictionary or not, if the cordinates are there, then I repeatedly add the current move and if not, I consider it to be unique and let process happen as is.

In [35]:
#Modified function

def Random_Walks_Line_Plot1(instances):

  # instances can be anything here, sequence of pi digits or some random numbers from 1 to 4 or 5 to 8 or any sequence.
  # using some random numbers just for the sake of tracing, but i am more interested in pi digits.
  # 'uniq' contains the unique values of the pi-digits dataset, which is 0 to 9
  uniq = np.unique(instances[:,0]).reshape(-1,1)
  n = len(uniq)
  # angles contains the angle at which the point has to be moved at a unit distance (depending upon how many unique values are there, those many angles are produced)
  angles = np.array([ int((x)*(360/n)) for x in range(len(uniq)) ]).reshape(-1,1)

  # uniq_angles is a dictionary that has pairs, value-angle : value can be any digit or number and angle represents the degree in which the movement has to be done.
  uniq_angles = dict(zip(uniq[:,0],angles[:,0]))
  
  # creating an empty 'x' array consisting of x and y co-ordinates and  Assigning the first point as origin.
  x = np.empty((len(instances)+1,2))
  x[0,0], x[0,1] = 0,0
  dct = {}

  # celluloid snippet to record the visualisations
  fig, ax = plt.subplots(dpi = 75)
  cam = Cam(fig)
  i = 1

  # iteration loop to go through instances, assign cordinates accordingly and plot them
  for k in instances[:,0]:
    angle = uniq_angles.get(k)

    # move to make from current x,y cordinate
    move = np.array([np.cos(np.radians(angle)),np.sin(np.radians(angle))])

    # move added to the current cordinate OR the new cordinate
    x[i] = x[i] + move

    # check if the new cordinate already exists, if it does, add the move again in hopes that until a new cordinate comes up that is not already in the dictionary of recorded cordinates
    while tuple(x[i]) in dct.items():
      x[i] = x[i] + move

    plt.plot(x[0:i,0], x[0:i,1], '.-')
    dct[x[i][0]] = x[i][1]
    trace = "instance : " + str(i) + " and value : " + str(k)  
    plt.axis('off')
    ax.text(0.1, 1.05, trace, transform=ax.transAxes, fontstyle = 'italic', fontweight = 'bold')
    cam.snap()
    i += 1
  plt.close(fig)
  animation = cam.animate(blit=False, interval=100)
  return animation, x
  

In [36]:
an, coords = Random_Walks_Line_Plot1(data[0:400])  
HTML(an.to_html5_video())

The above output is what is get, which i again feel is some kind of pattern that I am not really satisfied with. I am really sorry if i was not able to document it well enough but if we get on a call sometime, maybe i'll be able explain it more clear verbally.